# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

You may also want to implement:
- spell-checking for a concrete language - Russian, Tatar, etc. - any one you know, such that the solution accounts for language specifics,
- some recent (or not very recent) paper on this topic,
- solution which takes into account keyboard layout and associated misspellings,
- efficiency improvement to make the solution faster,
- any other idea of yours to improve the Norvig’s solution.

IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

> Implementation of basic class of spelling correction by Norvig

In [1]:
from collections import Counter

class SpellCorrector:
    """
    A class for spelling correction using a simple statistical model based on word frequencies.

    Attributes:
        counts (Counter): A Counter object containing word frequencies.
        words_cnt (int): Total count of words in the corpus.
    """
    def __init__(self, words):
        """
        Initialize the SpellCorrector object.

        Args:
            words (list): List of words to build the word frequency counter from.
        """
        self.counts = Counter(words)
        self.words_cnt = len(words)

    def correction(self, sentence): 
        """
        Return the most probable spelling correction for a sentence.

        Args:
            sentence (str): The input sentence to correct.
        Returns:
            str: The corrected sentence.
        """
        return ' '.join([max(self.candidates(word), key=lambda word: self.counts[word] / self.words_cnt) for word in sentence.lower().split()])

    def candidates(self, word):
        """
        Generate possible spelling corrections for a word.

        Args:
            word (str): The word to generate corrections for.
        Returns:
            set: A set of possible corrections for the word.
        """
        return (self.known([word]) or self.known(self.edits1(word)) or self.known(self.edits2(word)) or [word])

    def known(self, words):
        """
        Filter words to get the subset that appear in the dictionary.

        Args:
            words (list): List of words to filter.
        Returns:
            set: A set of words that appear in the dictionary.
        """
        return set(w for w in words if w in self.counts)

    def edits1(self, word):
        """
        Generate all edits that are one edit away from a word.

        Args:
            word (str): The input word.
        Returns:
            set: A set of words that are one edit away from the input word.
        """
        letters    = 'abcdefghijklmnopqrstuvwxyz'
        splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
        deletes    = [L + R[1:]               for L, R in splits if R]
        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
        replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
        inserts    = [L + c + R               for L, R in splits for c in letters]
        return set(deletes + transposes + replaces + inserts)

    def edits2(self, word):
        """
        Generate all edits that are two edits away from a word.

        Args:
            word (str): The input word.
        Returns:
            set: A set of words that are two edits away from the input word.
        """
        return (e2 for e1 in self.edits1(word) for e2 in self.edits1(e1))

# Small test to ensure that everything works as expected
true_words = ["spelling", "correct", "tests", "apple", "write", "process", 'test']
spell_corrector = SpellCorrector(true_words)

test_words = ["speling", "corect", "tets", "applo", "writ", "proces"]
for word in test_words:
    correction = spell_corrector.correction(word)
    print(f"{word}, {correction}")

speling, spelling
corect, correct
tets, test
applo, apple
writ, write
proces, process


> Implementation of spell checker with use of ngram model

In [2]:
import itertools
import pandas as pd

class NgramSpellCorrector(SpellCorrector):
    """
    A class for spelling correction using an n-gram language model.

    Attributes:
        n (int): The order of the n-gram model.
        ngram_file_path (str): The file path to the n-gram data.
        ngrams (dict): A dictionary containing n-grams as keys and their frequencies as values.
        counts (dict): A dictionary containing word frequencies.
    """
    def __init__(self, n=2, ngram_file_path='/kaggle/input/nlp-a2/bigrams.txt'):
        """
        Initialize the NgramSpellCorrector object.

        Args:
            n (int): The order of the n-gram model. Defaults to 2.
            ngram_file_path (str): The file path to the n-gram data. Defaults to '/kaggle/input/nlp-a2/bigrams.txt'.
        """
        ngrams = pd.read_csv(ngram_file_path, sep='\t', encoding='latin-1', header=None)
        self.ngrams = dict()
        self.counts = dict()
        self.n = n
        for items in ngrams.itertuples():
            key = tuple(items[i] for i in range(2, n + 2))
            self.ngrams[key] = int(items[1])
            for k in key:
                if k not in self.counts:
                    self.counts[k] = 0
                self.counts[k] += 1
        self.words_cnt = sum(self.ngrams.values())
        super().__init__(self.counts.keys())
    
    def correction(self, sentence):
        """
        Return the most probable spelling correction for a sentence using n-gram language model.

        Args:
            sentence (str): The input sentence to correct.
        Returns:
            str: The corrected sentence.
        """
        def P(x):
            if x not in self.ngrams:
                return 1 / (self.words_cnt + 2)
            return (self.ngrams[x] + 1) / (self.words_cnt + 2)
        
        words = sentence.lower().split(' ')
        candidates = []
        for word in words:
            if len(self.known([word])) == 1:
                candidates += [[word]]
                continue
            candidates += [self.candidates(word)]
        corrected = []
        combinations = list(itertools.product(*(candidates[i] for i in range(min(len(candidates), self.n)))))
        combinations.sort(key=P)
        corrected += [combinations[-1][i] for i in range(min(self.n, len(candidates)))]
        for i in range(self.n, len(candidates)):
            combinations = list(itertools.product([corrected[-1]], candidates[i]))
            combinations.sort(key=P)
            corrected += [combinations[-1][1]]
        return ' '.join(corrected)
    
    def candidates(self, word):
        """
        Generate possible spelling corrections for a word using n-gram language model.

        Args:
            word (str): The word to generate corrections for.
        Returns:
            set: A set of possible corrections for the word.
        """
        return (self.known(self.edits1(word)) or self.known(self.edits2(word)) or [word])
    
    def known(self, words):
        """
        Filter words to get the subset that appear in the dictionary.

        Args:
            words (list): List of words to filter.
        Returns:
            set: A set of words that appear in the dictionary.
        """
        return set(w for w in words if w in self.counts)

In [3]:
# Instance that utilize bigrams
bigram_corrector = NgramSpellCorrector()

# Moreover, I tested with 3-grams and 5-grams, but it has lower performance. I will show it in testing section

# corrector = NgramSpellCorrector(ngram_file_path='/kaggle/input/nlp-a2/wp_2gram.txt')
# corrector = NgramSpellCorrector(ngram_file_path='/kaggle/input/nlp-a2/coca_all_links.txt', n=3)
# corrector = NgramSpellCorrector(ngram_file_path='/kaggle/input/nlp-a2/fivegrams.txt', n=5)

> Small test to ensure that it's working

In [4]:
print(bigram_corrector.correction('wriqe biologyh expm'))
print(bigram_corrector.correction('I am goinq to killl all my enemes in gamee'))
print(bigram_corrector.correction('what a dog doin'))
print(bigram_corrector.correction('doing mth cooking mth telling mth'))

write biology expo
i am going to kill all my enemies in game
what a dog doin
doing math cooking th telling th


## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

### Justifications for Implementation Choices:

1. **N-gram Dataset Selection**:
   - I utilized the provided n-gram dataset `bigrams.txt`. This dataset was chosen because it provides the necessary n-gram frequencies required for building the language model. Moreover, I tried the Wikipedia Bigrams dataset, but it performed poorly. Therefore, I opted to stick with `bigrams.txt`.


2. **Word Skip**:
   - If a word is known by the spell checker, it will be skipped because it is already correctly spelled. We don't want to fix words that are correct according to their frequency.


3. **Incorporation of Laplace Smoothing**:
   - Laplace smoothing (add-one smoothing) was employed in computing n-gram probabilities to handle unseen n-grams in the dataset. This helps avoid zero probabilities for unseen n-grams and ensures that every n-gram has a non-zero probability.


4. **Beam Search Parameters**:
   - Beam search is not explicitly utilized. However, a form of beam search is implicitly performed when selecting the most probable correction for each word. Instead of exhaustively considering all possible combinations of corrections, I considered a subset of the most probable corrections based on the n-gram probabilities. This approach reduces computational complexity while still aiming to find a reasonable correction for the input sentence.

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity. Compare your solution to the Norvig's corrector, and report the accuracies.

In [5]:
test_df = pd.read_csv('/kaggle/input/spelling-mistake-data-1mn/test.csv')
test_df.head()

,text,augmented_text
0,project looks to mulesing genetic alternative,project looks to muelsnig ngeetic alternative
1,chemical agents used during protest at port au...,chemical agents used during LrotWst at port ah...
2,business chamber seeks budget infrastructure b...,business hcmaber seeks budget infrastrcutuer b...
3,3600 trips made to darwin tip after cyclone,3600 trips made to adrwni tip after cyconle
4,go between bridge to open july 5,go net3een brisye to lprn july 5


In [6]:
from tqdm import tqdm

def evaluate_correction(corrector, test_df, size=100):
    total_words = 0
    correct_words = 0
    total_sentences = 0
    correct_sentences = 0
    
    size = min(size, len(test_df))
    bar = tqdm(test_df[:size].iterrows(), total=size)
    for index, row in bar:
        original_text = row['text']
        augmented_text = row['augmented_text']
        corrected_text = corrector.correction(augmented_text)
        
        original_words = original_text.split()
        augmented_words = augmented_text.split()
        corrected_words = corrected_text.split()
        
#         total_words += len(original_words)
        total_words += sum(1 for orig, aug in zip(original_words, augmented_words) if orig != aug)
        total_sentences += 1
        
        if original_words == corrected_words:
            correct_words += len(original_words)
            correct_sentences += 1
        else:
            correct_words += sum(1 for orig, aug, corr in zip(original_words, augmented_words, corrected_words) if orig == corr and orig != aug)
    
    word_accuracy = correct_words / total_words if total_words > 0 else 0
    sentence_accuracy = correct_sentences / total_sentences if total_sentences > 0 else 0
    
    return word_accuracy, sentence_accuracy

In [8]:
words = []
for word in bigram_corrector.counts.keys():
    for i in range(bigram_corrector.counts[word]):
        words.append(word)

correctors = [(SpellCorrector(words), 'Norvig'), (bigram_corrector, 'Bigrams basic'), 
              (NgramSpellCorrector(ngram_file_path='/kaggle/input/nlp-a2/wp_2gram.txt'), 'Bigrams Wiki'),
              (NgramSpellCorrector(ngram_file_path='/kaggle/input/nlp-a2/coca_all_links.txt'), 'Trigrams'),
              (NgramSpellCorrector(ngram_file_path='/kaggle/input/nlp-a2/fivegrams.txt'), 'Fivegrams')]

In [9]:
for corrector, name in correctors:
    word_accuracy, sentence_accuracy = evaluate_correction(corrector, test_df, size=500)
    print(f"Metrics for '{name}'")
    print("\tWord Accuracy:", word_accuracy)
    print("\tSentence Accuracy:", sentence_accuracy)
    print()

100%|██████████| 500/500 [02:09<00:00,  3.85it/s]


Metrics for 'Norvig'
	Word Accuracy: 0.5463917525773195
	Sentence Accuracy: 0.116



100%|██████████| 500/500 [02:07<00:00,  3.93it/s]


Metrics for 'Bigrams basic'
	Word Accuracy: 0.6708961141950832
	Sentence Accuracy: 0.13



100%|██████████| 500/500 [01:51<00:00,  4.50it/s]


Metrics for 'Bigrams Wiki'
	Word Accuracy: 0.3203806502775575
	Sentence Accuracy: 0.058



100%|██████████| 500/500 [02:33<00:00,  3.26it/s]


Metrics for 'Trigrams'
	Word Accuracy: 0.36082474226804123
	Sentence Accuracy: 0.014



100%|██████████| 500/500 [02:24<00:00,  3.47it/s]

Metrics for 'Fivegrams'
	Word Accuracy: 0.49246629659000796
	Sentence Accuracy: 0.066



Bigram perform better others.